# 1. Loading prereqs

In [1]:
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running on Google Colab. ")
except:
    IN_COLAB = False
    print("Not running on Google Colab. ")

Not running on Google Colab. 


In [56]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    os.chdir("/content/drive/Shareddrives/AI4CYBSEC")

In [61]:
import os
os.getcwd()

'g:\\Drive condivisi\\AI4CYBSEC'

In [ ]:
NN1_WITH_DEFENCE=True
NN1_FLAG=False
NN2_FLAG=False

In [ ]:
if IN_COLAB:
    !pip install facenet-pytorch  # fornisce modelli pre-addestrati PyTorch per compiti di riconoscimento facciale
    !pip install Pillow # aggiunge il supporto per l'apertura, la manipolazione e il salvataggio di molti diversi formati di file immagine.


### 1.0 Load Model NN1

In [2]:
# utilizzo la libreria facenet_pytorch per caricare il modello InceptionResnetV1 preaddestrato sul dataset VGGFace2 e abilitare la classificazione.
from facenet_pytorch import InceptionResnetV1, MTCNN
import torch

resnet = InceptionResnetV1(pretrained='vggface2').eval()
resnet.classify = True

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))
resnet = resnet.to(device)

Running on device: cuda:0


In [3]:
import numpy as np
import tensorflow as tf

# Il modello è addestrato sulle seguenti Labels:
# Carico le labels del dataset VGGFACE
fpath = tf.keras.utils.get_file('rcmalli_vggface_labels_v2.npy',
                             "https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_labels_v2.npy",
                             cache_subdir="./")
LABELS = np.load(fpath) # List of name
# Clean list of name
for i in range(len(LABELS)):
  LABELS[i] = LABELS[i].strip().replace(' ', '').replace('"', '')


#### 1.1 Load Test Set

In [4]:
# set the path for the dataset

if IN_COLAB:
  path_dataset = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset"
else:
  path_dataset = "G:\Drive condivisi\AI4CYBSEC\\face_dataset"
identity_meta_NN1_name = "meta_identity_NN1.csv"

import pandas as pd
import os

path_identity_csv =os.path.join(path_dataset,identity_meta_NN1_name)
identity_meta_NN1 = pd.read_csv(path_identity_csv)

#### 1.2 Mapping label for NN1

In [5]:
# I want a dictonary related to the label of the Test Set that map the name of celebrities with label associated
name_to_id = {}
id_to_name = {}
for index, row in identity_meta_NN1.iterrows():
    # Ora puoi accedere ai valori di ogni riga come segue:
    class_id = row['Class_ID']
    name = row['Name']
    name_to_id[name]=class_id
    id_to_name[class_id] = name

In [6]:
from PIL import Image
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

class VGGFace2Dataset(Dataset):
    def __init__(self, root_dir, image_size=(160, 160), transform=None):
        self.root_dir = root_dir
        self.image_size = image_size
        self.transform = transform

        # List of files in the dataset
        self.file_list = []
        for root, dirs, files in os.walk(self.root_dir):
            for file in files:
                self.file_list.append(os.path.join(root, file))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path).resize(self.image_size)

        # Extract the label from the file path
        label = os.path.split(os.path.dirname(img_path))[-1]

        if self.transform:
            img = self.transform(img)

        return img, label

### 2.0 Definizione dataset NN1

In [ ]:
# Create transform for image resizing and normalization
data_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])
if IN_COLAB:
  test_set_path = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset/test_set_MTCNN"
else:
  test_set_path = "./face_dataset/test_set_MTCNN"
# Define dataset
dataset = VGGFace2Dataset(root_dir=test_set_path, transform=data_transform)
dataset_len = len(dataset)
# Check the length of the dataset
print("Dataset length:", dataset_len)
# Create DataLoader
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

Dataset length: 998


#### 2.1 Utility Function for NN1 with mapping labels

In [7]:

from PIL import Image
from torchvision import transforms
import torch
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

def load_image(file_path):
    """ carica un'immagine da un percorso e la apre come un'immagine utilizzando Image.open dal modulo Pillow.
    Successivamente, ridimensiona l'immagine a dimensioni 160x160 pixel e la converte in un tensore utilizzando
    transforms.ToTensor() dal modulo torchvision.transforms.
    Infine, restituisce sia il tensore dell'immagine che l'immagine aperta.
    """
    rsz = Image.open(file_path).resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    return tns, rsz
def make_inference(model, image_tensors, name_to_id, device):
    """
    Takes input image tensor and returns the label associated with the network's prediction.

    """
    # Move image tensors to the specified device
    image_tensors = image_tensors.to(device)

    probs = model(image_tensors)
    #print("probs", probs)

    # Get the number of elements along the first dimension
    num_elements = probs.size(0)

    # Initialize two lists to store the argmax
    argmax_list_1 = []
    argmax_list_2 = []

    # Compute argmax for each element along the first dimension
    for i in range(num_elements):
        target_class = np.array(probs[i].detach().cpu().numpy()).argmax()  # Move to CPU for numpy operations
        argmax_list_1.append(name_to_id[LABELS[target_class]])
        argmax_list_2.append(target_class)

    return argmax_list_1, argmax_list_2

def validate(dataset, model, name_to_id, device):
    """
    Validates a model on a dataset and returns the accuracy.

    Args:
        dataset: Dataloader to validate the model on.
        model: Model to validate.
        device: Device to perform inference on.

    Returns:
        accuracy: Accuracy of the model on the dataset.
    """
    model.eval()
    correct_predictions = 0
    total_samples = len(dataset) * dataset.batch_size

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in tqdm(dataset, desc="Validating model"):
            #images = mtcnn(images)
            predicted_classes, _ = make_inference(model, images, name_to_id, device)
            correct_predictions += sum(pred == label for pred, label in zip(predicted_classes, labels))

    # Compute accuracy
    accuracy = correct_predictions / total_samples
    return accuracy


def plot_image(original_image, original_label):
  """
  prende in ingresso le PIL.Image del campione originale e del corrispondete adversarial sample e li plotta
  """
  plt.figure()
  plt.matshow(original_image)
  plt.title("Model Prediction: {}".format(original_label))
  plt.show()

### 3.0 Validation on Clean Data on NN1


In [ ]:
acc = validate(dataloader, resnet, name_to_id, device)
print("\n Accuracy without defense "+str(acc))

Validating model: 100%|██████████| 998/998 [24:06<00:00,  1.45s/it]


 Accuracy without defense 0.8316633266533067


### 4.0 Implementing Deepfool Error Generic attack in ART

In [ ]:
if IN_COLAB:
  !pip install adversarial-robustness-toolbox[all] # installa la libreria ART

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3


In [8]:
import art
from art.estimators.classification import PyTorchClassifier
import torch.nn as nn
import numpy as np
import torch
from art.attacks.evasion import DeepFool

#### 4.1 Load list of images and labels

In [ ]:
# Attack.generete vuole dei numpy array, io ho dei tensori
# Recupero anche le true label dei 1000 campioni di test
#accuracy_clean_data = validate(dataloader, resnet, name_to_id, device)

images_list = []
labels_list = []
for image, label in dataloader:
    images_list.append(image.numpy())
    labels_list.append(label)


In [ ]:
print(len(images_list))

998


#### 4.2 Classifier for NN1

In [9]:
from torch import nn, optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters())
classifier = PyTorchClassifier(
    model=resnet,
    clip_values=(0,1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 160, 160),
    nb_classes=8631,
)

#### 4.3 Funzioni utili

In [10]:
import numpy as np
import csv
from datetime import datetime
import json
import matplotlib
import matplotlib.pyplot as plt

def compute_perturbation(original_images, adversarial_samples):
    perturbations = []
    for original_image, adversarial_sample in zip(original_images, adversarial_samples):
        perturbation = np.mean(np.abs((np.array(original_image) - np.array(adversarial_sample))))
        perturbations.append(perturbation)
    return round(float(np.mean(perturbations)),4)


def sec_curve(strength_values, accuracies_values, constant_values, strength_name, target_class=None,
              attack=None, avg_perturbations=None, accuracy_on_target_class=None, network = "NN1"):
    fig, ax = plt.subplots()
    # Costruisci la stringa per i valori costanti
    constant_str = ', '.join([f'{key}: {value}' for key, value in constant_values.items()])
    line = ax.plot(np.array(strength_values), np.array(accuracies_values), 'b--', label=f'{network} - {constant_str}')


    # Aggiungi i valori costanti come parte della legenda
    if target_class:
        plt.title('Security Curve for Target Class {}'.format(target_class))
    else:
        plt.title('Security Curve')

    # Aggiungi l'attacco al titolo
    if attack:
        plt.title(f'{attack} - {plt.gca().get_title()}')

    plt.xlabel('Attack strength ({})'.format(strength_name))
    plt.ylabel('Accuracy Test')
    plt.grid()

    # Aggiungi il diagramma a barre di colore arancione per avg_perturbations
    if avg_perturbations:
        x = np.array(strength_values)
        ax2 = ax.twinx()
        bar = ax2.bar(x, avg_perturbations, color='orange', alpha=0.5, width=0.01, label='Avg Perturbations')
        ax2.set_ylabel('Avg Perturbations')



    if accuracy_on_target_class:
        # Aggiungi la curva di accuratezza per la classe target
        ax.plot(np.array(strength_values), np.array(accuracy_on_target_class), 'r--', label=f'{network} - mis_targ/miss')

    # Unisci le linee e le barre in una lista per la legenda
    handles, labels = ax.get_legend_handles_labels()
    if avg_perturbations:
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles += handles2
        labels += labels2

    # Mostra la legenda
    #plt.legend(handles, labels, loc='upper right', shadow=True, fontsize='small')
    #voglio che la legenda sia fuori dal grafico
    plt.legend(handles, labels, loc='upper left', bbox_to_anchor=(1.2, 1), shadow=True, fontsize='small')
    plt.show()


def save_to_csv(attack_name, targeted, target_class, strength_name, strength_values, accuracy_values, constant_values, avg_perturbations, file_path, accuracy_on_target_class=None):
    # Intestazione del file CSV
    header = ["timestamp", "attacco", "targeted", "target_class", "strength_name", "strength_values", "accuracy_values", "constant_values", "avg_perturbations", "accuracy_on_target_class"]

    # Creazione della tupla con i valori da scrivere nel CSV
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Converti le liste di valori in stringhe JSON
    strength_values_json = json.dumps(strength_values)
    accuracy_values_json = json.dumps(accuracy_values)
    avg_perturbations_json = json.dumps(avg_perturbations)  # Converti avg_perturbations in una stringa JSON
    # Modifica: formatta strength_values come una lista di numeri invece di una stringa JSON
    accuracy_values_list = [float(val) for val in accuracy_values]
    # Modifica: converti accuracy_values in una lista di numeri
    row = (timestamp, attack_name, targeted, target_class, strength_name, strength_values_json, accuracy_values_list, json.dumps(constant_values), avg_perturbations_json)

    # Se accuracy_on_target_class è fornito e non è None, includilo nella tupla
    if accuracy_on_target_class is not None:
        accuracy_on_target_class_json = json.dumps(accuracy_on_target_class)  # Converti accuracy_on_target_class in una stringa JSON
        row += (accuracy_on_target_class_json,)
    else:
        row += (None,)  # Aggiungi None alla tupla

    # Scrittura nel file CSV in modalità append ('a')
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if file.tell() == 0:  # Se il file è vuoto, scrivi l'intestazione
            writer.writerow(header)
        writer.writerow(row)


def read_csv_and_plot(csv_file_path, network="NN1"):
    with open(csv_file_path, mode='r') as file:
        reader = csv.DictReader(file)

        for row in reader:
            timestamp = row["timestamp"]
            attack_name = row["attacco"]
            targeted = row["targeted"]
            target_class = row["target_class"]
            strength_name = row["strength_name"]
            strength_values = json.loads(row["strength_values"])
            accuracy_values = json.loads(row["accuracy_values"])
            constant_values = json.loads(row["constant_values"])
            avg_perturbations = json.loads(row["avg_perturbations"])
            accuracy_on_target_class = json.loads(row["accuracy_on_target_class"]) if row["accuracy_on_target_class"] else None

            # Controlla se accuracy_values è una stringa JSON e la elabora correttamente
            if isinstance(accuracy_values, str):
                accuracy_values = json.loads(accuracy_values)

            # Controlla se strength_values è una stringa JSON e la elabora correttamente
            if isinstance(strength_values, str):
                strength_values = json.loads(strength_values)

            # Controlla se strength_values è una stringa JSON e la elabora correttamente
            if isinstance(avg_perturbations, str):
                avg_perturbations = json.loads(avg_perturbations)

            # Chiama la funzione sec_curve per plottare la curva
            sec_curve(strength_values, accuracy_values, constant_values, strength_name, target_class, attack_name, avg_perturbations, accuracy_on_target_class, network)

### 2.2 SEC al variare di epsilon e max iter per NN1

In [ ]:
if IN_COLAB:
  results_csv = "/content/drive/Shareddrives/AI4CYBSEC/results/attack_results_NN1_deepfool.csv"
else:
  results_csv = "results\PGD\\attack_results_NN1.csv"
network = "NN1"

In [ ]:
from tqdm import tqdm
if NN1_FLAG:

  epsilon_range=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  max_iters = [2,5,10]

  for max_iter in max_iters:
    accuracy_test=list()
    accuracy_test.append(acc)
    pertubation_list=list()
    pertubation_list.append(0)
    for epsilon in tqdm (epsilon_range,desc="Generating DeepFool attacks"):
      attack = DeepFool(classifier=classifier, epsilon=epsilon, max_iter=max_iter, verbose=False)
      sample_indovinati=0
      images_adv_list=list()
      for img, label in zip(images_list, labels_list):
        test_image_adv_nn = attack.generate(img) #ritorna numpy array, per fare la valutazione ho bisogno di tensori
        images_adv_list.append(test_image_adv_nn)
        test_image_adv_tensor = torch.tensor(test_image_adv_nn)
        x_test_adv_pred,_ = make_inference(resnet, test_image_adv_tensor, name_to_id, device)

        if x_test_adv_pred[0] == label[0]:
                sample_indovinati += 1

      #ora ho finito di creare gli avd sample, posso calcolare l'accuracy e perturbation
      acc_eps=sample_indovinati/len(dataloader.dataset)
      accuracy_test.append(acc_eps)
      per=compute_perturbation(images_list, images_adv_list)
      pertubation_list.append(per)
      #SEC
    sec_curve([0]+epsilon_range, accuracy_test,{"max_iter":max_iter}, "eps", None,"DeepFool",pertubation_list,network="NN1")
      #Salvataggio risultati
    strength_name="eps" #valore che faccio cambiare
    constant_values = {"max_iter":max_iter}
    targeted=False
    attack_name="DeepFool"
    target_class=None
    save_to_csv(attack_name, targeted, target_class, strength_name, [0]+epsilon_range, accuracy_test, constant_values,pertubation_list,results_csv)

Generating DeepFool attacks:  33%|███▎      | 1/3 [04:03<08:06, 243.05s/it]

pertubation nn1 with eps = 0.1: 0.0018


Generating DeepFool attacks:  67%|██████▋   | 2/3 [07:55<03:57, 237.10s/it]

pertubation nn1 with eps = 0.2: 0.0019


Generating DeepFool attacks: 100%|██████████| 3/3 [11:51<00:00, 237.18s/it]

pertubation nn1 with eps = 0.3: 0.0021


###5.0 NN2

#### 5.1 Load NN2


#####5.1.1 Load repo for NN2

In [ ]:
# se non è presente la cartella VGGFACE2_pytorch  clone il repository
import os
if not os.path.exists('VGGFACE2_pytorch'):
    !git clone https://github.com/cydonia999/VGGFace2-pytorch.git
    !mv VGGFace2-pytorch VGGFACE2_pytorch

In [ ]:
import torch
import os

In [ ]:
%cd /content/drive/Shareddrives/AI4CYBSEC/

/content


In [ ]:
from VGGFace2_pytorch.models import senet as SENet
from VGGFace2_pytorch.models.resnet import resnet50 as ResNet
from VGGFace2_pytorch import utils
from VGGFace2_pytorch.trainer import Validator
from torch.utils.data import DataLoader
from VGGFace2_pytorch.datasets.vgg_face2 import VGG_Faces2
from torch.nn.modules.loss import CrossEntropyLoss

#####5.2 Load Model from pickel file

In [ ]:
import torchsummary

model = SENet.senet50(num_classes = 8631, include_top = True)
if IN_COLAB:
  weights_pickel = "/content/drive/Shareddrives/AI4CYBSEC/in_progress/senet50_ft_weight.pkl"
else:
  weights_pickel = ".\in_progress\senet50_ft_weight.pkl"

utils.load_state_dict(model, weights_pickel)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)

torchsummary.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13             [-1, 16, 1, 1]           4,112
             ReLU-14             [-1, 1

####5.3 Utility Function for NN2

In [ ]:
def preprocessing_on_tensor(img_tensor, mean_bgr=np.array([91.4953, 103.8827, 131.0912])):
    """
    Perform preprocessing on the input image tensor for the model.

    :param img_tensor: immagine with shape (C, H, W) and values in [0, 1]
    :return: immagine normalizzata using mean_bgr with shape (1, C, H, W)

    """
    img = img_tensor.squeeze(0)
    img = img.numpy()
    img = (img * 255).astype(np.uint8)
    mean_bgr = np.array([91.4953,103.8827, 131.0912])
    # img è C x H x W --> H x W x C
    img = np.transpose(img, (1, 2, 0))
    img = img[:, :, ::-1]  # RGB -> BGR
    img = img.astype(np.float32)
    img -= mean_bgr
    img = img.transpose(2, 0, 1)
    img_tensor = torch.from_numpy(img).unsqueeze(0)
    return img_tensor


def make_inference_NN2(model, img_tensor, device, with_preprocessing=True):
    """
    Esegue l'inferenza su un'immagine.
    :param model: modello
    :param img_tensor: immagine trasformata
    :param device: dispositivo
    :return: predizione
    """
    if with_preprocessing:
        img_tensor = img_tensor.squeeze(0)
        img_tensor = preprocessing_on_tensor(img_tensor)
    model.eval()
    img_tensor = img_tensor.to(device)

    with torch.no_grad():
        output = model(img_tensor)
        pred = torch.argmax(output, dim=1).item()
    return pred

####5.4 Mapping label for NN2

In [ ]:
import os
def create_image_list_file(root_dir, output_file, ext = '.jpg'):

    image_paths = []

    for class_id in os.listdir(root_dir):
        class_dir = os.path.join(root_dir, class_id)

        if os.path.isdir(class_dir):

            for filename in os.listdir(class_dir):

                if filename.endswith(ext):
                    image_path = f"{os.path.basename(root_dir)}/{class_id}/{filename}"
                    image_paths.append(image_path)

    with open(output_file, 'w') as f:
        for image_path in image_paths:
            f.write(image_path + '\n')

    print(f"File di output creato con successo: {output_file}")

In [ ]:
test_set_NN2 = "test_set_MTCNN_NN2"
if IN_COLAB:
    output_file = '/content/drive/Shareddrives/AI4CYBSEC/image_list_file_NN2.txt'
    meta_file = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset/identity_meta.csv"
else:
    output_file = 'image_list_file_NN2.txt'
    meta_file = ".\\face_dataset\identity_meta.csv"
root_dir = os.path.join(path_dataset,test_set_NN2)
create_image_list_file(root_dir, output_file)
id_label_dict = utils.get_id_label_map(meta_file)

File di output creato con successo: /content/drive/Shareddrives/AI4CYBSEC/image_list_file_NN2.txt


/content/drive/Shareddrives/AI4CYBSEC/VGGFace2_pytorch/utils.py:37: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(identity_list, sep=',\s+', quoting=csv.QUOTE_ALL, encoding="utf-8")


####5.6 Validate clean data NN2

In [ ]:
val_dataset = VGG_Faces2("./face_dataset", output_file, id_label_dict, split = 'valid') # dataset definito nella repo
val_loader = DataLoader(val_dataset, batch_size = 1)
validator = Validator(
            cmd = "test",
            cuda = True,
            model = model,
            criterion = CrossEntropyLoss(),
            val_loader = val_loader,
            log_file = "./log_file",
            print_freq = 1000,
          )

acc_NN2 = validator.validate()
print("Acc on clean data: ", acc_NN2)

Valid iteration=0 epoch=0:   0%|                        | 0/998 [00:00<?, ?it/s]

Test: [0/998/0]	epoch: 0	iter: 0	Time: 1.191 (1.191)	Loss: 0.0000 (0.0000)	Prec@1: 0.000 (0.000)	Prec@5: 0.000 (0.000)	


KeyboardInterrupt: 

####5.7 Validate NN2 on adv samples
Per valutare la trasferibilità degli attacchi. Attacco gray box.

In [ ]:
from torch import nn, optim

input_shape = (3,160,160)
nb_classes = 8631
## Attack Gray Box
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss = nn.CrossEntropyLoss()
#loss = nn.TripletMarginLoss()
optimizer = optim.Adam(resnet.parameters())
classifier = PyTorchClassifier(model=resnet, loss=loss, input_shape=input_shape, nb_classes=nb_classes, optimizer=optimizer, clip_values=(0, 1))

#####5.7.1 Load list of images and labels

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
  ])
if IN_COLAB:
  test_set_path = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset/test_set_MTCNN_NN2"
else:
  test_set_path = "./face_dataset/test_set_MTCNN_NN2"

# Define dataset
dataset = VGGFace2Dataset(root_dir=test_set_path, image_size=(224,224), transform=data_transform)

# Check the length of the dataset
print("Dataset length:", len(dataset))

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

images_list = []
labels_list = []
images_adv_list = []
for image, label in dataloader:
    image_numpy = image.numpy()
    images_list.append(image_numpy)
    labels_list.append(label)

Dataset length: 998


In [ ]:

network="NN2"
if IN_COLAB:
  results_csv = "/content/drive/Shareddrives/AI4CYBSEC/results/attack_results_NN2_deepfool.csv"
else:
  results_csv = "results\PGD\\attack_results_NN2.csv"
attack_name="DeepFool"

#####5.7.2 SEC at the change of esp and max_iter on NN2

In [ ]:
from tqdm import tqdm
from art.attacks.evasion import ProjectedGradientDescentPyTorch
import time

In [ ]:
if NN2_FLAG:
  epsilon_range=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
  max_iters = [2,5,10]

  for max_iter in max_iters:
    accuracy_test=list()
    accuracy_test.append(acc_NN2)
    pertubation_list=list()
    pertubation_list.append(0)
    for epsilon in tqdm (epsilon_range,desc="Generating DeepFool attacks"):
      attack = DeepFool(classifier=classifier, epsilon=epsilon, max_iter=max_iter, verbose=False)
      sample_indovinati=0
      images_adv_list=list()

      for img, label in zip(images_list, labels_list):
        test_image_adv_nn = attack.generate(img) #ritorna numpy array, per fare la valutazione ho bisogno di tensori
        images_adv_list.append(test_image_adv_nn)
        test_image_adv_tensor = torch.tensor(test_image_adv_nn)
        x_test_adv_pred,_ = make_inference_NN2(model, test_image_adv_tensor, device,with_preprocessing=True)

        if x_test_adv_pred[0] == id_label_dict[label[0]]:
          sample_indovinati += 1

      #ora ho finito di creare gli avd sample, posso calcolare l'accuracy e perturbation
      acc=sample_indovinati/len(dataloader.dataset)
      accuracy_test.append(acc)
      per=compute_perturbation(images_list, images_adv_list)
      pertubation_list.append(per)
    #SEC
    sec_curve([0]+epsilon_range, accuracy_test,{"max_iter":max_iter}, "eps", None,"DeepFool",pertubation_list,network="NN2")
    #save results
    strength_name="eps" #valore che faccio cambiare
    constant_values = {"max_iter":max_iter}
    targeted=False
    target_class=None
    save_to_csv(attack_name, targeted, target_class, strength_name,[0]+epsilon_range, accuracy_test,constant_values,pertubation_list, results_csv)

Generating DeepFool attacks:  20%|██        | 1/5 [03:51<15:25, 231.34s/it]

per eps = 0.5 ottengo perturbazione pari a 0.001


Generating DeepFool attacks:  40%|████      | 2/5 [07:45<11:39, 233.13s/it]

per eps = 0.6 ottengo perturbazione pari a 0.0011


Generating DeepFool attacks:  60%|██████    | 3/5 [11:35<07:42, 231.42s/it]

per eps = 0.7 ottengo perturbazione pari a 0.0011


Generating DeepFool attacks:  80%|████████  | 4/5 [15:26<03:51, 231.55s/it]

per eps = 0.8 ottengo perturbazione pari a 0.0012


Generating DeepFool attacks: 100%|██████████| 5/5 [19:22<00:00, 232.57s/it]

per eps = 0.9 ottengo perturbazione pari a 0.0013


####6.0 DeepFool NN1 with defense

#####6.1 Load Robus Detector for pre-processing

In [57]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if IN_COLAB:
    directory = "/content/drive/Shareddrives/AI4CYBSEC/models"
else:
    directory = ".\models"
def load_model(model, model_path):
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Definisci il modello mobilenet_v2
model = models.mobilenet_v2(pretrained=True)

# Sostituisci il classificatore dell'ultimo layer con un nuovo classificatore
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

model = model.to(device)

# carica i pesi del modello addestrato
defence = load_model(model, os.path.join(directory,'mobilenetv2_best_binary_classifier.pth'))

import torchsummary

# Stampa un riassunto del modello
torchsummary.summary(defence, (3, 160, 160))

c:\Program_for_ML\envs\AI4CyberSec\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Program_for_ML\envs\AI4CyberSec\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 80, 80]             864
       BatchNorm2d-2           [-1, 32, 80, 80]              64
             ReLU6-3           [-1, 32, 80, 80]               0
            Conv2d-4           [-1, 32, 80, 80]             288
       BatchNorm2d-5           [-1, 32, 80, 80]              64
             ReLU6-6           [-1, 32, 80, 80]               0
            Conv2d-7           [-1, 16, 80, 80]             512
       BatchNorm2d-8           [-1, 16, 80, 80]              32
  InvertedResidual-9           [-1, 16, 80, 80]               0
           Conv2d-10           [-1, 96, 80, 80]           1,536
      BatchNorm2d-11           [-1, 96, 80, 80]             192
            ReLU6-12           [-1, 96, 80, 80]               0
           Conv2d-13           [-1, 96, 40, 40]             864
      BatchNorm2d-14           [-1, 96,

#####6.2 Utility Functions

In [28]:
def make_inference_defence(model, img_tensor, device):
    #img must be a tensor with shape (N, C, H, W)
    model.eval()
    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()

def make_inference_NN1_with_defense(model,img_tensor, name_to_id, defense_model, device, isClean ):
    model.to(device)
    defense_model.to(device)
    prediction_defense = make_inference_defence(defense_model, img_tensor, device)
    if prediction_defense == 1:
        if not isClean:
            return 1, None
        return 0 ,None

    return make_inference(model, img_tensor, name_to_id, device)

def validate_with_defence(dataloader, model, name_to_id, device, defence_model, clean_data = True):
    """
    Validates a model on a dataset and returns the accuracy.

    Args:
        dataset: Dataloader to validate the model on.
        model: Model to validate.
        device: Device to perform inference on.

    Returns:
        accuracy: Accuracy of the model on the dataset.
    """
    model.to(device)
    model.eval()
    correct_predictions = 0
    total_samples = len(dataloader) * dataloader.batch_size
    num_skipped_samples = 0
    with torch.no_grad():  # Disable gradient calculation
        for images, labels in tqdm(dataloader, desc="Validating model"):
            predicted_classes, _= make_inference_NN1_with_defense(model, images, name_to_id, defence_model, device, clean_data)
            if predicted_classes == 1:# significa che ho predetto come adv un campione  adv
                num_skipped_samples += 1
                correct_predictions += 1
                continue
            elif predicted_classes == 0:# significa che ho predetto come adv un campione  clean
                num_skipped_samples += 1
                continue
            # DATO CHE C'è il continue è come se avessi un else:
            correct_predictions += sum(pred == label for pred, label in zip(predicted_classes, labels))


    # Compute accuracy
    accuracy = correct_predictions / total_samples
    return accuracy, num_skipped_samples

#####6.3 Validation on clean data with defense

In [58]:
# Create transform for image resizing and normalization

data_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])
if IN_COLAB:
  test_set_path = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset/test_set_MTCNN"
else:
  test_set_path = "./face_dataset/test_set_MTCNN"
  #test_set_path = "G:\Drive condivisi\AI4CYBSEC\\face_dataset\\adversarial_samples\DeepFool_v3"

# Define dataset
dataset = VGGFace2Dataset(root_dir=test_set_path, transform=data_transform)

# Check the length of the dataset
print("Dataset length:", len(dataset))

# Create DataLoader
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
acc = validate(dataloader, resnet, name_to_id, device)
print("\n Accuracy without defense "+str(acc))

acc_with_defence, num_skipped_samples = validate_with_defence(dataloader, resnet, name_to_id, device, defence, clean_data = True)
print("\n Accuracy with defense "+str(acc_with_defence))


Dataset length: 998


Validating model: 100%|██████████| 998/998 [00:22<00:00, 43.56it/s]



 Accuracy without defense 0.8346693386773547


Validating model: 100%|██████████| 998/998 [00:34<00:00, 29.26it/s]


 Accuracy with defense 0.8316633266533067


In [15]:
print("Numero di campioni saltati: ", num_skipped_samples)

Numero di campioni saltati:  8


##### 6.4 Deepfool attack with defence --NN1

######6.4.1 Load list of images and labels

In [37]:
images_list = []
labels_list = []
images_adv_list = []
for image, label in dataloader:
    # Effettua le predizioni del modello
    # image numpy on device
    image_numpy = image.numpy()
    images_list.append(image_numpy)
    labels_list.append(label)

print(images_list[0].shape)

(1, 3, 160, 160)


######6.4.2 Perform attack

In [20]:
if IN_COLAB:
  results_csv = "/content/drive/Shareddrives/AI4CYBSEC/results/attack_results_NN1_with_defense.csv"
else:
  results_csv = "results\\attack_results_NN1_with_defense.csv"

network = "NN1_with_defense"
attack_name="DeepFool"
targeted=False
target_class=None
NN1_WITH_DEFENCE=True

In [18]:
from tqdm import tqdm
from art.attacks.evasion import ProjectedGradientDescentPyTorch
import time

In [19]:
from torch import nn, optim
input_shape = (3,160,160)
nb_classes = 8631
## Attack Gray Box
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss = nn.CrossEntropyLoss()
#loss = nn.TripletMarginLoss()
optimizer = optim.Adam(resnet.parameters())
classifier = PyTorchClassifier(model=resnet, loss=loss, input_shape=input_shape, nb_classes=nb_classes, optimizer=optimizer, clip_values=(0, 1))

In [ ]:
if NN1_WITH_DEFENCE:
  epsilon_range=[0.1]
  max_iters = [2]
  for max_iter in max_iters:
    accuracy_test=[]
    accuracy_test.append(0.830)
    pertubation_list=[]
    pertubation_list.append(0)
    for epsilon in epsilon_range:
      attack = DeepFool(classifier=classifier, epsilon=epsilon, max_iter=max_iter, verbose=False)
      sample_indovinati=0
      num_skipped_samples=0
      images_adv_list=[]
      count_image=0
      for img, label in tqdm(zip(images_list, labels_list),total=len(images_list), desc=f"Generating DeepFool attacks with {epsilon} epsilon and {max_iter} max_iter"):
        test_image_adv_nn = attack.generate(img) #ritorna numpy array, per fare la valutazione ho bisogno di tensori
        images_adv_list.append(test_image_adv_nn)
        test_image_adv_tensor = torch.tensor(test_image_adv_nn)
        x_test_adv_pred,_ = make_inference_NN1_with_defense(resnet, test_image_adv_tensor, name_to_id,defence, device,isClean=False)
        if x_test_adv_pred == 1:# significa che ho predetto come adv un campione  adv
          num_skipped_samples += 1
          sample_indovinati += 1
          continue
        elif x_test_adv_pred == 0:# significa che ho predetto come adv un campione  clean
          num_skipped_samples += 1
          continue
        if x_test_adv_pred[0] == label[0]:
          sample_indovinati +=1
        count_image=count_image + 1
        if count_image == 200:
          break
      #ora ho finito di creare gli avd sample, posso calcolare l'accuracy e perturbation
      acc_eps=sample_indovinati/200
      accuracy_test.append(acc_eps)
      per=compute_perturbation(images_list[0:200], images_adv_list)
      pertubation_list.append(per)
      print("Values for eps = ",epsilon)
      print("accuracy: ", sample_indovinati/len(images_list))
      print("Number of skipped samples: ", num_skipped_samples)
      print("pertubation: ", per)

    #sec_curve([0] + epsilon_range, accuracy_test,{"max_iter":max_iter}, "eps", None,"DeepFool",pertubation_list,None, network)
    #Salvataggio risultati
    #strength_name="eps" #valore che faccio cambiare
    #constant_values = {"max_iter":max_iter}
    #save_to_csv(attack_name, targeted, target_class, strength_name,[0]+ epsilon_range, accuracy_test, constant_values,pertubation_list,results_csv)